# 1. LangChain basics - Messages, Prompts, Output Parsers and Chains

<a target="_blank" href="https://colab.research.google.com/github/IT-HUSET/ai-workshop-250121/blob/main/lab/1-langchain-basics.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a><br/>

## Setup

### Install dependencies

In [ ]:
%pip install python-dotenv~=1.0 docarray~=0.40.0 pydantic~=2.9 pypdf~=5.1 --upgrade --quiet
%pip install langchain~=0.3.7 langchain_openai~=0.2.6 langchain_community~=0.3.5 --upgrade --quiet
%pip install langchain-anthropic~=0.3.3 --upgrade --quiet

# If running locally, you can do this instead:
#%pip install -r ../requirements.txt

### Load environment variables

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# If running in Google Colab, you can use this code instead:
# from google.colab import userdata
# os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY")
# os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT")
# os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")

### Setup Models

In [ ]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
api_version = "2024-10-01-preview"
llm = AzureChatOpenAI(deployment_name="gpt-4o-mini", temperature=0.0, openai_api_version=api_version)


## LangChain basics

This notebook introduces some of the basics concepts of LangChain.

![LanChain](https://raw.githubusercontent.com/IT-HUSET/ai-workshop-250121/refs/heads/main/images/LangChain-chains.png)


### Chain, LCEL and the Runnable interface

A chain is a sequence of components with a unified interface, that are executed in order. This unified interface is called **[`Runnable`](https://python.langchain.com/docs/concepts/runnables/)** and provides common operations,  **invoking**, **streaming** and **batching** .

Multiple Runnables can be composed into a chain, where the output of one Runnable is passed as input to the next Runnable in the chain. The easiest way of doing this is by using the [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/concepts/lcel/), which basically simply is some syntactic sugar that allows components to be composed together using the `|` operator.

```python
chain = runnable1 | runnable2
```


The output of one runnable is passed as input to the next runnable in the chain.
https://python.langchain.com/docs/concepts/lcel/


### Chat models
LangChain provides a consistent interface for working with chat models from different providers. Read more [here](https://python.langchain.com/docs/concepts/chat_models/).


### Messages

Messages are the unit of communication in chat models. They are used to represent the input and output of a chat model, as well as any additional context or metadata that may be associated with a conversation.

![Graph](https://github.com/IT-HUSET/ai-workshop-250121/blob/main/images/langchain-messages.png?raw=true)

Read more about messages [here](https://python.langchain.com/docs/concepts/messages/).


### Output parsing

Output parsers are responsible for taking the output of a model and transforming it to a more suitable format for downstream tasks.

Read more [here](https://python.langchain.com/docs/concepts/output_parsers/)


### More LangChain concepts

Read more about basic LangChain concepts [here](https://python.langchain.com/docs/concepts/).



## Let's start Chat models and Messages

We define a system message and a human message to start a conversation

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage

# We define a system message and a human message to start a conversation
system_message = SystemMessage(content="You are a helpful assistant, expert in Iceland tourist information.")
human_message = HumanMessage(content="Hi! I need help planning a trip to Iceland.")

#### Since Chat Models are Runnables, we can invoke them using the `invoke` method

In [ ]:
ai_message: AIMessage = llm.invoke([system_message, human_message])
print(ai_message) # This will (basically) print the entire response from the LLM, including a lot of meta-data, metrics, etc.

In [ ]:
# Print just the content of the AI message
print(ai_message.content)

#### Conversation

In [ ]:
# Let's try a follow-up question
conversation_messages: list[BaseMessage] = [
    system_message,
    human_message,
    ai_message,
    HumanMessage(content="What if there's a volcanic eruption!?😱")
]

response = llm.invoke(conversation_messages)
print(response.content)

Let's try with a different LLM

In [ ]:
from langchain_anthropic import ChatAnthropic

llm2 = ChatAnthropic(
     model='claude-3-5-sonnet-20241022',
     temperature=0.0,
)

response = llm2.invoke(conversation_messages)
print(response.content)

----
<br/>

## Prompts and Prompt Templates

### Chat prompt template - for chat-based LLMs
Basically a chat prompt template is a list of message templates. The result of invoking a chat prompt template is a `ChatPromptValue`, containing a list of messages.

```python
ChatPromptValue(
    messages=[
        SystemMessage(content='You are a helpful AI bot. Your name is Carl.'),
        HumanMessage(content='Hello, there!'),
    ]
)
```

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_template = "Translate user input into a style that is {style}."

# This is the easiest and most common way to create a prompt template
prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("human", "{input}"), # You can also use the alias "user" instead of "human"
])

#### Let's check the messages templates

In [ ]:
print(prompt_template.messages)

#### Print input variables

In [ ]:
print(prompt_template.input_variables)

### Using a prompt template

In [ ]:
from langchain_core.prompt_values import ChatPromptValue

customer_style = "American English in a calm and respectful tone"

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages: ChatPromptValue = prompt_template.invoke({'style': customer_style, 'input': customer_email})

#### Let's have a look at the contents (messages):

In [ ]:
# First (system) message:
print(type(customer_messages.messages[0]))
print(customer_messages.messages[0].content)

In [ ]:
# Second (human) message:
print(type(customer_messages.messages[1]))
print(customer_messages.messages[1].content)

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = llm.invoke(customer_messages)
print(customer_response.content)

#### Try another example

In [ ]:
customer_style = "The same style as the user"

customer_messages: ChatPromptValue = prompt_template.invoke({'style': customer_style, 'input': customer_email})

customer_response = llm.invoke(customer_messages)
print(customer_response.content)

----
<br/>


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

## The most common parser - String output parsing

Useful when you just want to extract a string (content) from the output, and not all the other metadata an LLM might return.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()
response = llm.invoke("Copenhagen or Many-worlds?")
print(response) # Will print out the entire response, a lot of which we don't need

In [ ]:
# Let's just extract the content
parsed_response = str_parser.invoke(response)
print(parsed_response)

## Structured output parsing

![Stryctyred output](https://python.langchain.com/assets/images/structured_output-2c42953cee807dedd6e96f3e1db17f69.png)

#### This is what we'd like the output to look like:

In [ ]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "pretty affordable!"
}

#### Setup the inputs and prompt template

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any quote about the value or price.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

#### Let's try it out

In [ ]:
messages = prompt_template.format_messages(text=customer_review)
response = llm.invoke(messages)
print(response.content)
print(f"\nResponse type: {type(response.content)}")

### Parse the LLM output as JSON

Let's fix this with proper JSON parsing

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()

result = json_parser.invoke(response)
print(result)
print(f"\nResponse type: {type(result)}")

### Structured parsing with typing and optional validation (using **Pydantic**)

In [ ]:
from pydantic import BaseModel

class Review(BaseModel):
    gift: bool
    delivery_days: int
    price_value: str


structured_output_llm = llm.with_structured_output(Review)
result = structured_output_llm.invoke(messages)

print(result)
print(f"\nResponse type: {type(result)}")
print(f"Delivery days: {result.delivery_days}")


----
<br/>

## Chains

### Simple Chain

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

# When using only a single template string, it's assumed the role is "human"
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

print(prompt)
output_parser = StrOutputParser()

In [ ]:
# Build a chain (creates a RunnableSequence)
chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"topic": "bears"})

### Streamed response

In [ ]:
for chunk in chain.stream({"topic": "bears"}):
    print(chunk, end="|", flush=True)


### Bind with parameters

#### Temperature

In [ ]:
temp_model = llm.bind(temperature=1.0) | StrOutputParser()
temp_model.invoke("Can you give me three great tips about what to do in Reykjavik?")

#### Stop words

In [ ]:
stop_model = llm.bind(stop=["Harpa"]) | StrOutputParser()
stop_model.invoke("Can you give me three great tips about what to do in Reykjavik?")